# 04 - H2O AutoML 企业级应用

> 🎯 学习目标：掌握H2O AutoML的企业级特性，理解分布式训练和模型可解释性

---

## 📚 本节内容

1. H2O AutoML 简介
2. 基础使用
3. 排行榜分析
4. 模型可解释性
5. 分布式训练
6. 模型部署

---

## 🏢 H2O AutoML 企业级特性

### 核心优势

1. **企业级功能** 🏭
   - 分布式训练
   - 生产级部署
   - 模型监控

2. **可解释性强** 🔍
   - SHAP值分析
   - 变量重要性
   - 部分依赖图

3. **Web界面** 🌐
   - Flow UI可视化
   - 无代码操作

4. **模型导出** 📦
   - MOJO/POJO格式
   - 跨平台部署

---

## 1. 安装与初始化

In [ ]:
# 安装 H2O
# !pip install h2o
# 注意：需要 Java 8+ 环境

import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np

# 初始化 H2O
h2o.init(
    max_mem_size='4G',    # 最大内存
    nthreads=-1           # 使用所有CPU核心
)

print("✅ H2O 初始化成功")
print(h2o.cluster().show_status())

---

## 2. 分类任务基础使用

### 2.1 加载数据

In [ ]:
# 从URL加载数据
data_url = "https://s3.amazonaws.com/h2o-public-test-data/smalldata/iris/iris_wheader.csv"
data = h2o.import_file(data_url)

# 查看数据
print("数据形状:", data.shape)
print("\n前5行:")
data.head()

### 2.2 数据准备

In [ ]:
# 设置目标变量和特征
target = 'class'
features = data.columns
features.remove(target)

# 将目标变量转换为因子（分类）
data[target] = data[target].asfactor()

# 划分训练集和测试集
train, test = data.split_frame(ratios=[0.8], seed=42)

print(f"训练集: {train.shape[0]} 行")
print(f"测试集: {test.shape[0]} 行")

### 2.3 训练 H2O AutoML

In [ ]:
# 创建 AutoML 对象
aml = H2OAutoML(
    max_runtime_secs=300,      # 最大运行时间（秒）
    max_models=20,             # 最多训练的模型数
    seed=42,
    nfolds=5,                  # 交叉验证折数
    sort_metric='AUTO'         # 自动选择排序指标
)

# 训练
print("开始训练（最多5分钟）...")
aml.train(
    x=features,
    y=target,
    training_frame=train
)

print("\n✅ 训练完成！")

---

## 3. 排行榜分析

### 3.1 查看模型排行榜

In [ ]:
# 查看所有模型的排行榜
lb = aml.leaderboard

print("模型排行榜:")
lb.head(rows=lb.nrows)

### 3.2 最佳模型详情

In [ ]:
# 获取最佳模型
best_model = aml.leader

print("最佳模型信息:")
print(f"模型ID: {best_model.model_id}")
print(f"\n模型类型: {best_model.algo}")

# 查看模型摘要
print("\n模型摘要:")
best_model.summary()

### 3.3 在测试集上评估

In [ ]:
# 预测
predictions = best_model.predict(test)

print("预测结果（前10行）:")
predictions.head(10)

In [ ]:
# 评估性能
performance = best_model.model_performance(test)

print("测试集性能:")
print(performance)

---

## 4. 模型可解释性 ⭐

### 4.1 变量重要性

In [ ]:
# 查看变量重要性
var_imp = best_model.varimp(use_pandas=True)

print("变量重要性:")
print(var_imp)

# 可视化
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(var_imp['variable'], var_imp['relative_importance'])
plt.xlabel('Relative Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

### 4.2 部分依赖图 (Partial Dependence Plot)

In [ ]:
# 对某个特征绘制部分依赖图
pdp = best_model.partial_plot(
    data=train,
    cols=['sepal_len'],
    plot=True
)

### 4.3 SHAP值分析（如果支持）

In [ ]:
# 对于支持的模型，可以计算SHAP值
# 注意：不是所有H2O模型都支持SHAP

try:
    # 计算SHAP值
    shap_values = best_model.predict_contributions(test)
    print("SHAP值（前5行）:")
    shap_values.head(5)
except:
    print("该模型不支持SHAP值计算")

---

## 5. 回归任务示例

In [ ]:
# 加载回归数据集
boston_url = "https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/BostonHousing.csv"
boston = h2o.import_file(boston_url)

# 设置目标和特征
target_reg = 'medv'
features_reg = boston.columns
features_reg.remove(target_reg)

# 划分数据
train_reg, test_reg = boston.split_frame(ratios=[0.8], seed=42)

# 训练AutoML
aml_reg = H2OAutoML(
    max_runtime_secs=180,
    seed=42
)

print("开始回归任务训练...")
aml_reg.train(x=features_reg, y=target_reg, training_frame=train_reg)

# 评估
perf_reg = aml_reg.leader.model_performance(test_reg)
print("\n回归模型性能:")
print(perf_reg)

---

## 6. 模型保存与部署

### 6.1 保存模型

In [ ]:
# 保存最佳模型
model_path = h2o.save_model(
    model=best_model,
    path='./h2o_models',
    force=True
)

print(f"✅ 模型已保存到: {model_path}")

### 6.2 加载模型

In [ ]:
# 加载保存的模型
loaded_model = h2o.load_model(model_path)

# 使用加载的模型预测
loaded_predictions = loaded_model.predict(test)

print("✅ 模型加载成功")
print("\n预测结果（前5行）:")
loaded_predictions.head(5)

### 6.3 导出为 MOJO（生产部署）

In [ ]:
# 导出为 MOJO 格式（跨平台部署）
mojo_path = best_model.download_mojo(
    path='./h2o_models',
    get_genmodel_jar=True  # 下载运行MOJO所需的jar文件
)

print(f"✅ MOJO已导出到: {mojo_path}")
print("\n可以在生产环境中使用MOJO进行快速推理")

---

## 7. 高级特性

### 7.1 自定义训练参数

In [ ]:
# 高级配置
aml_advanced = H2OAutoML(
    max_runtime_secs=300,
    max_models=20,
    
    # 停止条件
    stopping_metric='AUTO',
    stopping_tolerance=0.001,
    stopping_rounds=3,
    
    # 交叉验证
    nfolds=5,
    
    # 保持交叉验证的预测
    keep_cross_validation_predictions=True,
    
    # 保持交叉验证的模型
    keep_cross_validation_models=False,
    
    seed=42
)

print("✅ 高级配置创建成功")

### 7.2 限制算法类型

In [ ]:
# 只使用树模型
aml_trees = H2OAutoML(
    max_runtime_secs=180,
    include_algos=['GBM', 'XGBoost', 'DRF'],  # GBM, XGBoost, Random Forest
    seed=42
)

print("配置为只使用树模型")

---

## 8. 总结与最佳实践

### H2O AutoML 核心优势

1. **企业级**: 分布式训练，生产部署
2. **可解释**: SHAP值，变量重要性，部分依赖图
3. **灵活**: Web界面 + Python API
4. **高性能**: 自动集成，stacking

### 使用场景

```python
# 快速验证（3-5分钟）
H2OAutoML(max_runtime_secs=180)

# 标准训练（15-30分钟）
H2OAutoML(max_runtime_secs=1800)

# 深度搜索（1-3小时）
H2OAutoML(max_runtime_secs=10800)
```

### 最佳实践

1. **数据准备**: 使用 `h2o.import_file()` 或 `h2o.H2OFrame()`
2. **因子转换**: 分类任务记得 `asfactor()`
3. **内存管理**: 合理设置 `max_mem_size`
4. **模型部署**: 使用MOJO进行生产部署
5. **可解释性**: 利用变量重要性和部分依赖图

### 优势与局限

**优势**：
- ✅ 企业级功能完善
- ✅ 分布式训练支持
- ✅ 模型可解释性强
- ✅ 部署便捷（MOJO）

**局限**：
- ❌ 需要Java环境
- ❌ 学习曲线稍陡
- ❌ 内存占用较大

---

## 🎯 练习任务

1. **任务1**: 对比H2O AutoML和sklearn的性能
2. **任务2**: 分析模型排行榜，理解不同模型的特点
3. **任务3**: 使用SHAP值分析模型决策
4. **任务4**: 导出MOJO并在生产环境测试

---

## 🧹 清理资源

In [ ]:
# 关闭H2O集群
h2o.cluster().shutdown()

print("✅ H2O集群已关闭")

---

## 📚 参考资源

- [H2O AutoML 官方文档](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html)
- [H2O Flow UI](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/flow.html)
- [MOJO部署指南](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/productionizing.html)

---

**下一节**: `05_automl_comparison.ipynb` - AutoML工具对比实验